![alt text](https://anvaqta.id/headerai.jpg "Compression.dl")

# Character Sequence Text Generation

Di sini kita akan membuat language model untuk membangkitkan text dari level karakter berdasarkan input sekuens karakter yang diberikan

In [0]:
!pip install tensorflow-gpu

In [0]:
import tensorflow as tf

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Reshape
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import get_file

import numpy as np
import random
import sys
import io

# Text Data
Untuk memulainya, kita perlu memiliki data untuk melatih model kita. Anda dapat menggunakan file teks apa pun yang Anda inginkan untuk proses ini

di sini telah disediakan beberapa data text yang bisa digunakan

In [0]:
dataset = {
    'shakespeare'  : 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
    'wonderland'   : 'https://www.gutenberg.org/cache/epub/11/pg11.txt',
    'harry'        : 'https://www.linguistik.uzh.ch/dam/jcr:169bff5c-ac13-457b-9acb-4fe7f1ad5cb0/Harry%20Potter%20and%20the%20Sorcerer.txt',
    'nietzsche'    : 'https://s3.amazonaws.com/text-datasets/nietzsche.txt',
    'frankenstein' : 'https://www.gutenberg.org/files/84/84-0.txt',
    'liriklagu'    : 'https://raw.githubusercontent.com/dhamirdesrul/Text-Generation/master/lagupeterpan.txt'
}

Pilih satu data

In [0]:
# inisilisasi variable filename dengan file dataset
filename = ?? 
path = get_file( filename.split('/')[-1], origin=filename)


# **Preprocessing**

Semua data akan dilakukan prapemrosesan, data tersebut akan dibersihkan seperti: 
- Menghapus karakter yang tidak diperlukan
- lowercase

In [0]:
# import regex
??

kalimat = ''
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

# melakukan preprocessing dengan menghilangkan "'", "\n", ".", "-", ","
kalimat = ?? 
text = kalimat
text[:250]

# Encoding
Jaringan saraf bekerja dengan angka, bukan karakter teks. Jadi kita perlu mengkonversi input karakter menjadi angka. 

Pertama, kita urutkan daftar unik semua karakter yang muncul dalam teks tersebut, kemudian gunakan fungsi enumerasi untuk mendapatkan angka yang mewakili karakter tersebut. 

Berikutnya buat kamus yang menyimpan kunci dan nilai, atau karakter dan angka yang mewakili mereka.

In [0]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = ??
indices_char = ??
# indices_char

# Sequence Building

Di sini kita set bahwa maksimum sequence dari karakter input adalah 40

Untuk itu, kita harus memotong semua text dalam bentuk sekuens semi-redundan sepanjang 40 karakter. Kita gunakan nilai redundansi sebesar 3 karakter

artinya, misal kita memiliki teks: `"saya suka makan nasi"`, kemudian kita buat sekuens semi-redundan dengan panjang 5 dan redundansi 2, maka kita akan memiliki
* `'saya '` dengan target `'aya s'`
* `'ya su'` dengan target `'a suk'`
* `' suka'` dengan target `'suka '`
* `'uka m'` dengan target `'ka ma'`
* dan seterusnya


In [0]:
# cut the text in semi-redundant sequences of maxlen characters
?? 
?? 
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    # memasukkan text ke dalam sentences
    ?? 
    
    #memasukkan text selanjutnya ke dalam next_chars
    ?? 
    
print('nb sequences:', len(sentences))



In [0]:
for i in range(10):
  print([sentences[i]],[next_chars[i]])

Berikutnya kita buat data latih dan targetnya berupa vektor angka yang diambil dari dictionary berdasarkan kalimat sekuens yang sudah kita buat

In [0]:
# print('Vectorization...')
# menginisialisasi x dan y berisikan matriks kosong sebesar 3x3 menggunakan sentences, maxlen, dan chars

x = ?? 
y = ?? 


# melibatkan hasil encoding diatas
# mengisi x dengan nilai 1 berdasarkan jumlah index kalimat, index karakter, index karakter dalam char_indicies
??
        

# mengisi y dengan nilai 1 berdasarkan jumlah index kalimat selanjutnya, index karakter selanjutnya, jumlah karakter selanjutnya
??
        


# LSTM Model
Sekarang kita coba bangun jaringan sederhana mengguankan 1 layer LSTM dengan ukuran output vektor 128. Setelah layer LSTM, kita tambahkan Layer Dense untuk memprediksi kelanjutan karakter dari 40 karakter input

Many to Many RNN

![alt text](https://indoml.files.wordpress.com/2018/03/kind-many-to-many2.jpg)

In [0]:
#inisialisasi model dengan squential
model = Sequential()
# menambahkan model dengan LSTM sebanyak 256, input_shape diisi maxlen, panjang karakter, dan parameter return_sequences
model.add(LSTM(??))
model.add(LSTM(??))

# menambahkan model dengan dense sebanyak panjang karakter dan aktivasi menggunakan softmax
model.add(TimeDistributed(Dense(??)))

#menambahkan model dengan melakukan reshape dengan maxlen dan panjang karakter
model.add(Reshape((??)))

#melakukan optimizer menggunakan 'adam' dengan learning rate sebesar 0.01
optimizer = Adam(lr=0.01)
# model akan di compile menggunakan parameter loss 'categorical_crossentropy' dan optimizer 'adam'
model.compile(??)

In [0]:
# Menampilkan layer yang telah dibuat
from tensorflow.keras.utils import plot_model
plot_model(model)

# Sample Probability Function
Berikut adalah helper function untuk melakukan sampling karakter output berdasarkan output probability dari softmax

In [0]:
def sample(preds, temperature=1.0):
  
  """
    
    Inputs:
    - preds: prediksi model
    - temperature: temperatur diisi dengan 1.0
    
    Outputs:
    Nilai probabilitas 
    """
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


# Training Checkpoint
Berikutnya mari kita tambahkan sebuah callback pada fungsi training agar kita bisa melihat contoh hasil pembangkitan text yang dilakukan setiap 5 epoch

In [0]:
def on_epoch_end(epoch, _):
  if epoch%5==0:
    # Function invoked at end of each epoch. Prints generated text.
    print('\n---------------------------------------------------------------------')
    print('>>>>> Generating text after Epoch: %d' % epoch)

    # mencari nilai random dari index 0 sampai jumlah text-maxlen-1
    start_index = random.randint(??) 
    diversity = 0.7
    print('\n>>>>> diversity:', diversity)

    generated = ''
    # menginisialisasi sentence pada text dengan index [start_index: start_index + maxlen]
    sentence = text[start_index: start_index + maxlen]
    # menambahkan variable generate dengan value sentence
    generated += sentence
    print('>>>>> Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    # input nilai a berdasarkan jumlah output karakter yang diinginkan
    a = 100
    for i in range(a):
        # menginisiasi x_pred menggunakan matrix 0 dengan ukuran 1x maxlen x panjang karakter
        x_pred = np.zeros((1, maxlen, len(chars)))
        # mengisi x_pred dengan nilai 1 berdasarkan index [0, index karakter, dan jumlah karakter]
        for t, char in enumerate(sentence):
            x_pred[??] = ??.
        
        # menginisialisasi preds dengan hasil prediksi model dengan parameter x_pred dan verbose=0 indeks 0
        preds = model.predict(x_pred, verbose=0)[0]
        # menginisialisasi next_index dengan sampling karakter
        next_index = sample(preds[-1], diversity)
        # menginisialisasi next_char dengan karakter berdasarkan index dalam indicies_char index [next_char]
        next_char = indices_char[next_index]
        # menambahkan generated dengan  value next_char
        generated += next_char
        # menginisiasi sentence dengan value sentences dari index [1: ] ditambahkan dengan value next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print('\n---------------------------------------------------------------------')
    print('>>>>> Continuing training')
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Training Process

Sekarang tinggal kita latih model Text Generator kita

In [0]:
model.fit(x, y,
          batch_size=512,
          epochs=50,
          callbacks=[print_callback])

# Testing Process
setelah model terlatih, mari kita uji untuk membangkitkan text sepanjang 400 karakter

In [0]:
input_kata = input()
index_kata = text.index(input_kata)
index_kata

In [0]:
# start_index = random.randint(0, len(text) - maxlen - 1)
diversity = 0.7
print('\n>>>>> diversity:', diversity)

#menginisialisasi generated dengan string kosong
generated = ''
#menginisialisasi sentence denga text berdasarkan index [index_kata: index_kata + maxlen]
sentence = text[index_kata: index_kata + maxlen]
#menambahkan generated dengan value sentence
generated += sentence
print('>>>>> Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

# input nilai a berdasarkan jumlah output karakter yang diinginkan
a = 200
for i in range(a):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    # menginisialisasi preds dengan hasil prediksi model dengan parameter x_pred dan verbose=0 indeks 0
    preds = model.predict(x_pred, verbose=0)[0]
    # menginisialisasi next_index dengan sampling karakter
    next_index = sample(preds[-1], diversity)
    # menginisialisasi next_char dengan karakter berdasarkan index dalam indicies_char index [next_char]
    next_char = indices_char[next_index]
    
    # menambahkan generated dengan  value next_char
    generated += next_char
    # menginisiasi sentence dengan value sentences dari index [1: ] ditambahkan dengan value next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()


<p>Copyright &copy; 2019 <a href=https://www.linkedin.com/in/andityaarifianto/>ADF</a> </p>